In [ ]:
from dorna2 import Dorna
from camera import Camera
from dorna_vision import Detection
from dorna_vision import grasp
from dorna2 import pose as dorna_pose
import tools
import config
import time
import numpy as np
import math

#####################################################################
# Connect to the robot:
robot = Dorna()
robot_connected = False
connection_try_counter = 0

while not robot_connected:
    if robot.connect(config.robot["ip"]):
        print("Robot connected")
        robot_connected = True
    else:
        print("Robot connection failed")

    if not robot_connected:
        connection_try_counter += 1
        time.sleep(1)
        if connection_try_counter == 100:
            print("Maximum connection attempts reached")

# Connect and initialize the camera
camera = Camera()
try:
    camera.connect()
except Exception as e:
    print(f"Camera connection failed: {e}")
else:
    print("Camera connected")

ROI_corners_vials = [[226, 18], [624, 18], [624, 470], [226, 470]]
ROI_corners_caps = [[251, 64], [593, 64], [593, 424], [251, 424]]
# Ind_ROI_corners = [[409, 103], [554, 225], [432, 378], [283, 256]]

# Detections:
preset_vials = {'roi': {'corners': ROI_corners_vials, 'inv': False, 'crop': True, 'offset': 0}, "camera_mount":{"type": "dorna_ta_j4","ej": [0 ,0, 0, 0, 0, 0, 0, 0],"T": [48.05923462271373, 30.4362801279415, -5.585734144443932, 0.4111487682507738, -0.18114590496686764, 90.05010215826098]}, 'display': {'label': 0, 'save_img': True, 'save_img_roi': True}, 'detection': {'cmd': 'od', 'path': 'auto_sampler_vial_inner.pkl', 'conf': 0.65, 'cls': []}}
detection_vials = Detection(camera=camera, robot=robot, **preset_vials)

preset_caps = {'roi': {'corners': ROI_corners_caps, 'inv': False, 'crop': True, 'offset': 0}, 'detection': {'cmd': 'od', 'path': 'vial_cap.pkl', 'conf': 0.65, 'cls': []}, "camera_mount":{"type": "dorna_ta_j4","ej": [0 ,0, 0, 0, 0, 0, 0, 0],"T": [48.05923462271373, 30.4362801279415, -5.585734144443932, 0.4111487682507738, -0.18114590496686764, 90.05010215826098]}, 'display': {'label': 0, 'save_img': True, 'save_img_roi': True}}
detection_caps = Detection(camera=camera, robot=robot, **preset_caps)

# preset_individual = {'roi': {'corners': Ind_ROI_corners, 'inv': False, 'crop': True, 'offset': 0}, "camera_mount":{"type": "dorna_ta_j4","ej": [0 ,0, 0, 0, 0, 0, 0, 0],"T": [48.05923462271373, 30.4362801279415, -5.585734144443932, 0.4111487682507738, -0.18114590496686764, 90.05010215826098]}, 'display': {'label': 0, 'save_img': False, 'save_img_roi': True}, 'detection': {'cmd': 'od', 'path': 'auto_sampler_vial.pkl', 'conf': 0.8, 'cls': []}}
# detection_individual = Detection(camera=camera, robot=robot, **preset_individual)

#Go to initial position:
# tools.safe_init(config=config, robot=robot)
tools.output(config.robot_gripper["open"], robot)
tools.output(config.decapper["open"], robot)

tools.take_init_picture_caps(config=config, robot=robot) #This should be included in the safe init motion
print("Joints before picture: ", robot.get_all_joint())
print("Target joints: ", config.camera["joints_caps"])

retval = detection_caps.run()
print("Caps detected: ", len(retval))

cap_vis_pxls = []

for x in retval:
    pxl = x["center"]
    cap_vis_pxls.append(pxl)

Caps_coords = tools.generate_cap_coords_vision(config=config, cap_vis_pxls=cap_vis_pxls)
print("Interpolated cap world coords from vision with griddata: ")
for x in Caps_coords: print(x)

tools.take_init_picture(config=config, robot=robot) #This should be included in the safe init motion
print("Joints before picture: ", robot.get_all_joint())
print("Target joints: ", config.camera["joints_vials"])

# Call the detection
retval = detection_vials.run()
print("Vials detected: ", len(retval))

# camera_data = detection_vials.get_camera_data()

# Initialize control variables
# Calculate the position of the robot's x offset in the world frame
tools.update_robot_frame(config=config)

# vials_coords_vision = []
vial_vis_pxls = []
ids = []

for x in retval:
    # Correct xyz:
    # detection_xyz = x["xyz"]
    pxl = x["center"]
    id = x["id"]
    vial_vis_pxls.append(pxl)
    ids.append(id)

    # xyz = tools.correct_vision_xyz(config=config, pxl=pxl, camera=camera, camera_data=camera_data, detection=detection_general, target="vials")
    # xyz_world = tools.translate_vis_to_world(config=config, xyz=xyz)
    # vials_coords_vision.append(xyz_world)

print("Vial pxls from vision: ")
for x in vial_vis_pxls: print(x)

print("Ids:")
print(ids)

vial_coords_from_grid = tools.generate_vial_coords_vision(config=config, vial_vis_pxls=vial_vis_pxls)
print("Vial coords:")
print(vial_coords_from_grid)
# print("Interpolated vial world coords from vision with griddata: ")
# for x in vial_coords_from_grid: print(x)

sorted_vials = tools.sort_vials(config=config, points=vial_coords_from_grid)
print("Sorted world coords from vision with griddata: ")
for x in sorted_vials: print(x)

sorted_ids = []
for x in sorted_vials:
    sorted_ids.append(ids[vial_coords_from_grid.index(x)])

print("Ids sorted to match sorted vials: ", sorted_ids)

vials_to_do = min(len(sorted_vials), len(Caps_coords))
print("Vials to do: ", vials_to_do)
ind = 4

print("Initialization completed")

init_drop_rail = 485 #Original: 450
ref_drop_world = 771.95 #Taken from a real run, could also be: 770.19
rail_diff = ref_drop_world - init_drop_rail

camera.close()

for i in range(vials_to_do):
    if i < ind:
        continue
    elif i > ind:
        ind = i 

    start_time = time.time()
    tools.update_cap_frame(config=config, cap_frame=Caps_coords[ind], rail_x=435)

    xyz_to_use = sorted_vials[ind]
    new_rail = xyz_to_use[0] - rail_diff #Done to ensure that all drops use the same robot X coordinate and reduce joint changes

    tools.update_vial_frame_vision(config=config, vial_frame_vision=xyz_to_use, rail_x=new_rail)

    rvec_base = config.camera["frame"][3:]
    # best_pick
    best_rvec_pick = grasp.collision_free_rvec(
        sorted_ids[ind],
        rvec_base, 
        config.robot_gripper["gripper_opening"], config.robot_gripper["finger_wdith"], config.robot_gripper["finger_location"], 
        detection_vials, mask_type="elp", prune_factor=2, num_steps=360)        

    if best_rvec_pick is None:
        print("no grasp found")
        config.vial_holder["pick"][3:] = [0.23287556085751854, -0.3172811067209202, 47.98701517310126]
    else:
        print("best rvec pick: ", best_rvec_pick)

        init_mat = robot.kinematic.axis_angle_to_mat(u=rvec_base.copy())
        init_mat_inv = np.linalg.inv(init_mat)
        target_mat = robot.kinematic.axis_angle_to_mat(u=best_rvec_pick.copy())
        trans_mat = init_mat_inv * target_mat

        rot_in_init_rot = robot.kinematic.mat_to_axis_angle(mat=trans_mat.copy())
        print("rot: ", rot_in_init_rot)
        config.vial_holder["pick"][3:] = rot_in_init_rot

    #Sequence:
    tools.vial_tray_to_decapper(config=config, robot=robot)
    tools.cap_to_vial(config=config, robot=robot)
    tools.capping(config=config, robot=robot)
    tools.vial_to_tray(config=config, robot=robot)
    end_time = time.time()

    print("Vial processing completed")
    print("Pick and place movement execution time: ", end_time-start_time)
    # break #To test a single vial

print("Sequence completed")

[[745.2339969487297, -232.7298866169822],[744.6685952980828, -246.81463413651036],[743.6566879449164, -260.37809967485674],[730.3208810419686, -232.83755843390398],[730.5862609805829, -247.38382480567444],[730.7175649403548, -260.23356378420937],[716.3687276125695, -232.11593066797005],[716.703376524562, -245.7622531758908],[716.2847044888811, -259.7943206739224]]

Robot connected
Camera connected
Joints before picture:  [-37.792969, 6.152344, -110.852051, 0.043945, 14.677734, -74.992676, 485.04375, 0.0]
Target joints:  [-37.792969, 6.152344, -110.830078, 0.043945, 14.655762, -74.86084]
Caps detected:  9
Interpolated cap world coords from vision with griddata: 
[889.0852044434918, -163.03206311691193]
[914.7180743708786, -165.4727543572508]
[888.643020840022, -190.9683752548307]
[914.0250261776844, -189.6177333348359]
[862.1652294473569, -139.44403934866085]
[888.2841554144611, -138.26100046662265]
[861.9540650533794, -163.87273639126926]
[862.4306497130559, -191.51024332755378]
[915.2277988058225, -139.53000422630845]
Joints before picture:  [-72.949219, 1.867676, -120.541992, 0.021973, 28.696289, -72.817383, 485.00625, 0.0]
Target joints:  [-72.817383, 1.867676, -120.498047, 0.021973, 28.608398, -72.817383]
Vials detected:  9
Vial pxls from vision: 
[391, 254]
[507, 198]
[451, 225]
[406, 212]
[422, 170]
[493, 241]
[463, 182]
[437, 268]
[478, 28

KeyboardInterrupt: 

In [7]:
for x in detection_vials.retval["valid"]:
    print(x)

target_id = 1
target_tvec = [d["xyz"] for d in detection_vials.retval["valid"] if d["id"] == target_id][0]
print(target_tvec)

{'timestamp': 1761240923.556753, 'cls': 'vial_inner', 'conf': 0.8881635665893555, 'center': [539, 256], 'corners': [[529, 246], [548, 246], [548, 265], [529, 265]], 'color': (33, 255, 255), 'id': 0, 'ej': [0, 0, 0, 0, 0, 0, 0, 0], 'xyz': [56.528310719203155, -244.67191368132958, -50.25221575659276]}
{'timestamp': 1761240923.556753, 'cls': 'vial_inner', 'conf': 0.8820018768310547, 'center': [411, 212], 'corners': [[402, 202], [421, 202], [421, 221], [402, 221]], 'color': (33, 255, 255), 'id': 1, 'ej': [0, 0, 0, 0, 0, 0, 0, 0], 'xyz': [100.31704985976266, -246.33652361784542, -53.84520627349343]}
{'timestamp': 1761240923.556753, 'cls': 'vial_inner', 'conf': 0.8776702880859375, 'center': [452, 83], 'corners': [[442, 74], [462, 74], [462, 93], [442, 93]], 'color': (33, 255, 255), 'id': 2, 'ej': [0, 0, 0, 0, 0, 0, 0, 0], 'xyz': [99.62678924657013, -289.3315737439216, -48.41006695236456]}
{'timestamp': 1761240923.556753, 'cls': 'vial_inner', 'conf': 0.85919189453125, 'center': [482, 426], 'c

In [81]:
from dorna2 import pose as dorna_pose
# grasp
rvec_base = [180, 0, 0]
gripper_opening = 15.5 # mm,
finger_wdith = 6.5 # mm 
finger_location = [0, 180]
gripper_rotation = [
    {"axis": [0, 0, 1], "angle": 0},
    {"axis": [0, 0, 1], "angle": 180}
]
r = 4
# best_pick
best_rvec_pick = grasp.collision_free_rvec(
    # r["id"], 
    r,
    rvec_base, 
    gripper_opening, finger_wdith, finger_location, 
    detection_vials, mask_type="elp", prune_factor=2, num_steps=360)        

if best_rvec_pick is None:
    print("no grasp found")
else:
    print(best_rvec_pick)

init_mat = robot.kinematic.axis_angle_to_mat(u=rvec_base.copy())
init_mat_inv = np.linalg.inv(init_mat)
target_mat = robot.kinematic.axis_angle_to_mat(u=best_rvec_pick.copy())
trans_mat = init_mat_inv * target_mat

rot_in_init_rot = robot.kinematic.mat_to_axis_angle(mat=trans_mat.copy())
print("rot: ", rot_in_init_rot)

init_rot = rvec_base
# new_orient = [127.37959233, 52.82123032, 180]
new_orient = [-2.277857235209531e-15, -5.505375072421775e-15, 89.90959803683529]

pour_rvec_first = robot.kinematic.rotate_rvec(rvec=init_rot, axis=[1,0,0], angle=new_orient[0], local=True)
pour_rvec_second = robot.kinematic.rotate_rvec(rvec=pour_rvec_first, axis=[0,1,0], angle=new_orient[1], local=True)
pour_rvec_thrid = robot.kinematic.rotate_rvec(rvec=pour_rvec_second, axis=[0,0,1], angle=new_orient[2], local=True)
print(pour_rvec_thrid)

print(vial_coords_from_grid)

# for r in detection_vials.retval["valid"]:
#     r["tvec"] = xyz_interp.astype(float).tolist()[0]

pose_all = [vial_coords_from_grid[r] + dorna_pose.rotate_abc(best_rvec_pick, axis=rotation["axis"], angle=rotation["angle"], local=True) for rotation in gripper_rotation]
for x in pose_all: print(x)

[70.20086848232107, 165.74630633690714, 0.0]
rot:  [-5.439181999081233e-15, 8.210669004214776e-15, -134.0904007576938]
[127.37959233054521, -127.17876968151606, 0.0]
[[716.2847044888811, -259.7943206739224], [744.6685952980828, -246.81463413651036], [716.703376524562, -245.7622531758908], [730.5862609805829, -247.38382480567444], [745.2339969487297, -232.7298866169822], [716.3687276125695, -232.11593066797005], [743.6566879449164, -260.37809967485674], [730.7175649403548, -260.23356378420937], [730.3208810419686, -232.83755843390398]]
[745.2339969487297, -232.7298866169822, 70.20086801142212, 165.7463052251009, -0.0]
[745.2339969487297, -232.7298866169822, 165.74630633690714, -70.200868482321, 0.0]


In [ ]:
def generate_vial_drop(config, final_loc, tilt_angle, alignment_angle=45):
    init_rot = init_pose[3:]
    rvec_first = robot.kinematic.rotate_rvec(rvec=init_rot, axis=[0,1,0], angle=-tilt_angle, local=False) #Rotation to insert vial
    rvec_second = robot.kinematic.rotate_rvec(rvec=rvec_first, axis=[1,0,0], angle=-tilt_angle, local=False) #Rotation to insert vial
    rvec_final = robot.kinematic.rotate_rvec(rvec=rvec_second, axis=[0,0,1], angle=-alignment_angle, local=False) #Rotation to avoid other vials
    config.vial_holder["drop"][3:] = rvec_final
    print(config.vial_holder["drop"])

init_pose = [178.653739, -217.879284, 50.71865, 180, 0, 0]
init_rot = init_pose[3:]

tilt_angle = 45
pour_rvec_first = robot.kinematic.rotate_rvec(rvec=init_rot, axis=[0,1,0], angle=-tilt_angle, local=False) #Rotation to pout the water
pour_rvec_second = robot.kinematic.rotate_rvec(rvec=pour_rvec_first, axis=[1,0,0], angle=-tilt_angle, local=False) #Rotation to pout the water
pour_rvec_final = robot.kinematic.rotate_rvec(rvec=pour_rvec_second, axis=[0,0,1], angle=-45, local=False) #Included to aligned bottle pouring to gripper front

# pour_rvec_final = robot.kinematic.rotate_rvec(rvec=init_rot, axis=[0,0,1], angle=-45, local=False)

print("Bottle pour angle updated")

print(init_pose)
print(init_rot)
print(pour_rvec_final)

final_loc = [1.2848547803481551, -0.6376452704425901, -14.334862947201302, -1.2390190108418174e-06, 1.5174821696505201e-06, 45.000000000000014]
final_orient = final_loc[3:]

[0, tilt_angle, 0]

init_rot = [180, 0, 0]

tilt_angle2 = 45
fist_rot = robot.kinematic.rotate_rvec(rvec=init_rot, axis=[1,0,0], angle=final_orient[0], local=True)
second_rot = robot.kinematic.rotate_rvec(rvec=fist_rot, axis=[0,1,0], angle=final_orient[1], local=True)
third_rot = robot.kinematic.rotate_rvec(rvec=second_rot, axis=[0,0,1], angle=final_orient[2], local=True)
fourth_rot = robot.kinematic.rotate_rvec(rvec=third_rot, axis=[0,1,0], angle=tilt_angle2, local=True)

print(third_rot)
print(fourth_rot)



Bottle pour angle updated
[178.653739, -217.879284, 50.71865, 180, 0, 0]
[180, 0, 0]
[119.01645745459449, -26.962097726556486, 26.962097726556443]
[166.29831699962298, -68.88301505518544, 2.682209014892578e-06]
[140.78179362364665, -58.31372589587072, 58.313730914363]
